In [2]:
import pandas as pd

df = pd.read_csv('raw_dat_1.csv')

columns_to_delete = list(df.columns[:3]) + list(df.columns[8:])
df = df.drop(columns=columns_to_delete, axis=1)
df = df.iloc[:-400]

output_file = 'raw_dat_v2.csv'
df.to_csv(output_file, index=False)

In [3]:
from math import sqrt

import torch
import torch.nn as nn
from torch import Tensor

import os
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings('ignore')

import argparse
from torch import nn, optim
from torch.optim import lr_scheduler
from tqdm import tqdm

#from models import Autoformer, DLinear, TimeLLM

#from data_provider.data_factory import data_provider
import time
import random

import matplotlib.pyplot as plt
import shutil

plt.switch_backend('agg')

class Normalize(nn.Module):
    def __init__(self, num_features: int, eps=1e-5, affine=False, subtract_last=False, non_norm=False):
        """
        :param num_features: the number of features or channels
        :param eps: a value added for numerical stability
        :param affine: if True, RevIN has learnable affine parameters
        """
        super(Normalize, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.affine = affine
        self.subtract_last = subtract_last
        self.non_norm = non_norm
        if self.affine:
            self._init_params()

    def forward(self, x, mode: str):
        if mode == 'norm':
            self._get_statistics(x)
            x = self._normalize(x)
        elif mode == 'denorm':
            x = self._denormalize(x)
        else:
            raise NotImplementedError
        return x

    def _init_params(self):
        # initialize RevIN params: (C,)
        self.affine_weight = nn.Parameter(torch.ones(self.num_features))
        self.affine_bias = nn.Parameter(torch.zeros(self.num_features))

    def _get_statistics(self, x):
        dim2reduce = tuple(range(1, x.ndim - 1))
        if self.subtract_last:
            self.last = x[:, -1, :].unsqueeze(1)
        else:
            self.mean = torch.mean(x, dim=dim2reduce, keepdim=True).detach()
        self.stdev = torch.sqrt(torch.var(x, dim=dim2reduce, keepdim=True, unbiased=False) + self.eps).detach()

    def _normalize(self, x):
        if self.non_norm:
            return x
        if self.subtract_last:
            x = x - self.last
        else:
            x = x - self.mean
        x = x / self.stdev
        if self.affine:
            x = x * self.affine_weight
            x = x + self.affine_bias
        return x

    def _denormalize(self, x):
        if self.non_norm:
            return x
        if self.affine:
            x = x - self.affine_bias
            x = x / (self.affine_weight + self.eps * self.eps)
        x = x * self.stdev
        if self.subtract_last:
            x = x + self.last
        else:
            x = x + self.mean
        return x

class Dataset_Custom(Dataset):
    def __init__(self, root_path, flag='train', size=None,
                 features='S', data_path='ETTh1.csv',
                 target='OT', scale=True,
                 #timeenc=0, freq='h',
                 percent=100
                 #seasonal_patterns=None
                 ):
        if size == None:
            self.seq_len = 24 * 4 * 4
            self.label_len = 24 * 4
            self.pred_len = 24 * 4
        else:
            self.seq_len = size[0]
            self.label_len = size[1]
            self.pred_len = size[2]
        # init
        assert flag in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[flag]

        self.features = features
        self.target = target
        self.scale = scale
        #self.timeenc = timeenc
        #self.freq = freq
        self.percent = percent

        self.root_path = root_path
        self.data_path = data_path
        self.__read_data__()

        self.enc_in = self.data_x.shape[-1]
        self.tot_len = len(self.data_x) - self.seq_len - self.pred_len + 1

    def __read_data__(self):
        self.scaler = StandardScaler()
        df_raw = pd.read_csv(os.path.join(self.root_path,
                                          self.data_path))

        '''
        df_raw.columns: ['date', ...(other features), target feature]
        '''
        cols = list(df_raw.columns)
        cols.remove(self.target)
        #cols.remove('date')
        #df_raw = df_raw[['date'] + cols + [self.target]]
        df_raw = df_raw[cols + [self.target]]
        num_train = int(len(df_raw) * 0.7)
        num_test = int(len(df_raw) * 0.2)
        num_vali = len(df_raw) - num_train - num_test
        border1s = [0, num_train - self.seq_len, len(df_raw) - num_test - self.seq_len]
        border2s = [num_train, num_train + num_vali, len(df_raw)]
        border1 = border1s[self.set_type]
        border2 = border2s[self.set_type]

        if self.set_type == 0:
            border2 = (border2 - self.seq_len) * self.percent // 100 + self.seq_len

        if self.features == 'M' or self.features == 'MS':
            cols_data = df_raw.columns[1:]
            df_data = df_raw[cols_data]
        elif self.features == 'S':
            df_data = df_raw[[self.target]]

        if self.scale:
            train_data = df_data[border1s[0]:border2s[0]]
            self.scaler.fit(train_data.values)
            data = self.scaler.transform(df_data.values)
        else:
            data = df_data.values

        '''df_stamp = df_raw[['date']][border1:border2]
        df_stamp['date'] = pd.to_datetime(df_stamp.date)
        if self.timeenc == 0:
            df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
            df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
            df_stamp['weekday'] = df_stamp.date.apply(lambda row: row.weekday(), 1)
            df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)
            data_stamp = df_stamp.drop(['date'], 1).values
        elif self.timeenc == 1:
            data_stamp = time_features(pd.to_datetime(df_stamp['date'].values), freq=self.freq)
            data_stamp = data_stamp.transpose(1, 0)
        '''
        self.data_x = data[border1:border2]
        self.data_y = data[border1:border2]
        #self.data_stamp = data_stamp

    def __getitem__(self, index):
        feat_id = index // self.tot_len
        s_begin = index % self.tot_len

        s_end = s_begin + self.seq_len
        r_begin = s_end - self.label_len
        r_end = r_begin + self.label_len + self.pred_len
        seq_x = self.data_x[s_begin:s_end, feat_id:feat_id + 1]
        seq_y = self.data_y[r_begin:r_end, feat_id:feat_id + 1]
        #seq_x_mark = self.data_stamp[s_begin:s_end]
        #seq_y_mark = self.data_stamp[r_begin:r_end]

        return seq_x, seq_y #, seq_x_mark, seq_y_mark

    def __len__(self):
        return (len(self.data_x) - self.seq_len - self.pred_len + 1) * self.enc_in

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)

def data_provider(args, flag):
    Data = Dataset_Custom
    #timeenc = 0 if args.embed != 'timeF' else 1
    percent = args.percent

    if flag == 'test':
        shuffle_flag = False
        drop_last = True
        batch_size = args.batch_size
        #freq = args.freq
    else:
        shuffle_flag = True
        drop_last = True
        batch_size = args.batch_size
        #freq = args.freq

    data_set = Data(
            root_path=args.root_path,
            data_path=args.data_path,
            flag=flag,
            size=[args.seq_len, args.label_len, args.pred_len],
            features=args.features,
            target=args.target,
            #timeenc=timeenc,
            #freq=freq,
            percent=percent
            #seasonal_patterns=args.seasonal_patterns
        )
    data_loader = DataLoader(
        data_set,
        batch_size=batch_size,
        shuffle=shuffle_flag,
        num_workers=args.num_workers,
        drop_last=drop_last)
    return data_set, data_loader

class TokenEmbedding(nn.Module):
    def __init__(self, c_in, d_model):
        super(TokenEmbedding, self).__init__()
        padding = 1 if torch.__version__ >= '1.5.0' else 2
        self.tokenConv = nn.Conv1d(in_channels=c_in, out_channels=d_model,
                                   kernel_size=3, padding=padding, padding_mode='circular', bias=False)
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_in', nonlinearity='leaky_relu')

    def forward(self, x):
        x = self.tokenConv(x.permute(0, 2, 1)).transpose(1, 2)
        return x


class ReplicationPad1d(nn.Module):
    def __init__(self, padding) -> None:
        super(ReplicationPad1d, self).__init__()
        self.padding = padding

    def forward(self, input: Tensor) -> Tensor:
        replicate_padding = input[:, :, -1].unsqueeze(-1).repeat(1, 1, self.padding[-1])
        output = torch.cat([input, replicate_padding], dim=-1)
        return output


class PatchEmbedding(nn.Module):
    def __init__(self, d_model, patch_len, stride, dropout):
        super(PatchEmbedding, self).__init__()
        # Patching
        self.patch_len = patch_len
        self.stride = stride
        self.padding_patch_layer = ReplicationPad1d((0, stride))

        # Backbone, Input encoding: projection of feature vectors onto a d-dim vector space
        self.value_embedding = TokenEmbedding(patch_len, d_model)

        # Positional embedding
        # self.position_embedding = PositionalEmbedding(d_model)

        # Residual dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # do patching
        n_vars = x.shape[1]
        x = self.padding_patch_layer(x)
        x = x.unfold(dimension=-1, size=self.patch_len, step=self.stride)
        x = torch.reshape(x, (x.shape[0] * x.shape[1], x.shape[2], x.shape[3]))
        # Input encoding
        x = self.value_embedding(x)
        return self.dropout(x), n_vars

#from math import sqrt

#import torch
#import torch.nn as nn

from transformers import LlamaConfig, LlamaModel, LlamaTokenizer, GPT2Config, GPT2Model, GPT2Tokenizer, BertConfig, \
    BertModel, BertTokenizer
#from layers.Embed import PatchEmbedding
import transformers
#from layers.StandardNorm import Normalize

transformers.logging.set_verbosity_error()


class FlattenHead(nn.Module):
    def __init__(self, n_vars, nf, target_window, head_dropout=0):
        super().__init__()
        self.n_vars = n_vars
        self.flatten = nn.Flatten(start_dim=-2)
        self.linear = nn.Linear(nf, target_window)
        self.dropout = nn.Dropout(head_dropout)

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear(x)
        x = self.dropout(x)
        return x


class Model(nn.Module):

    def __init__(self, configs, patch_len=16, stride=8):
        super(Model, self).__init__()
        self.task_name = configs.task_name
        self.pred_len = configs.pred_len
        self.seq_len = configs.seq_len
        self.d_ff = configs.d_ff
        self.top_k = 5
        self.d_llm = configs.llm_dim
        self.patch_len = configs.patch_len
        self.stride = configs.stride

        if configs.llm_model == 'LLAMA':
            # self.llama_config = LlamaConfig.from_pretrained('/mnt/alps/modelhub/pretrained_model/LLaMA/7B_hf/')
            self.llama_config = LlamaConfig.from_pretrained('huggyllama/llama-7b')
            self.llama_config.num_hidden_layers = configs.llm_layers
            self.llama_config.output_attentions = True
            self.llama_config.output_hidden_states = True
            try:
                self.llm_model = LlamaModel.from_pretrained(
                    # "/mnt/alps/modelhub/pretrained_model/LLaMA/7B_hf/",
                    'huggyllama/llama-7b',
                    trust_remote_code=True,
                    local_files_only=True,
                    config=self.llama_config,
                    # load_in_4bit=True
                )
            except EnvironmentError:  # downloads model from HF is not already done
                print("Local model files not found. Attempting to download...")
                self.llm_model = LlamaModel.from_pretrained(
                    # "/mnt/alps/modelhub/pretrained_model/LLaMA/7B_hf/",
                    'huggyllama/llama-7b',
                    trust_remote_code=True,
                    local_files_only=False,
                    config=self.llama_config,
                    # load_in_4bit=True
                )
            try:
                self.tokenizer = LlamaTokenizer.from_pretrained(
                    # "/mnt/alps/modelhub/pretrained_model/LLaMA/7B_hf/tokenizer.model",
                    'huggyllama/llama-7b',
                    trust_remote_code=True,
                    local_files_only=True
                )
            except EnvironmentError:  # downloads the tokenizer from HF if not already done
                print("Local tokenizer files not found. Atempting to download them..")
                self.tokenizer = LlamaTokenizer.from_pretrained(
                    # "/mnt/alps/modelhub/pretrained_model/LLaMA/7B_hf/tokenizer.model",
                    'huggyllama/llama-7b',
                    trust_remote_code=True,
                    local_files_only=False
                )
        elif configs.llm_model == 'GPT2':
            self.gpt2_config = GPT2Config.from_pretrained('openai-community/gpt2')

            self.gpt2_config.num_hidden_layers = configs.llm_layers
            self.gpt2_config.output_attentions = True
            self.gpt2_config.output_hidden_states = True
            try:
                self.llm_model = GPT2Model.from_pretrained(
                    'openai-community/gpt2',
                    trust_remote_code=True,
                    local_files_only=True,
                    config=self.gpt2_config,
                )
            except EnvironmentError:  # downloads model from HF is not already done
                print("Local model files not found. Attempting to download...")
                self.llm_model = GPT2Model.from_pretrained(
                    'openai-community/gpt2',
                    trust_remote_code=True,
                    local_files_only=False,
                    config=self.gpt2_config,
                )

            try:
                self.tokenizer = GPT2Tokenizer.from_pretrained(
                    'openai-community/gpt2',
                    trust_remote_code=True,
                    local_files_only=True
                )
            except EnvironmentError:  # downloads the tokenizer from HF if not already done
                print("Local tokenizer files not found. Atempting to download them..")
                self.tokenizer = GPT2Tokenizer.from_pretrained(
                    'openai-community/gpt2',
                    trust_remote_code=True,
                    local_files_only=False
                )
        elif configs.llm_model == 'BERT':
            self.bert_config = BertConfig.from_pretrained('google-bert/bert-base-uncased')

            self.bert_config.num_hidden_layers = configs.llm_layers
            self.bert_config.output_attentions = True
            self.bert_config.output_hidden_states = True
            try:
                self.llm_model = BertModel.from_pretrained(
                    'google-bert/bert-base-uncased',
                    trust_remote_code=True,
                    local_files_only=True,
                    config=self.bert_config,
                )
            except EnvironmentError:  # downloads model from HF is not already done
                print("Local model files not found. Attempting to download...")
                self.llm_model = BertModel.from_pretrained(
                    'google-bert/bert-base-uncased',
                    trust_remote_code=True,
                    local_files_only=False,
                    config=self.bert_config,
                )

            try:
                self.tokenizer = BertTokenizer.from_pretrained(
                    'google-bert/bert-base-uncased',
                    trust_remote_code=True,
                    local_files_only=True
                )
            except EnvironmentError:  # downloads the tokenizer from HF if not already done
                print("Local tokenizer files not found. Atempting to download them..")
                self.tokenizer = BertTokenizer.from_pretrained(
                    'google-bert/bert-base-uncased',
                    trust_remote_code=True,
                    local_files_only=False
                )
        else:
            raise Exception('LLM model is not defined')

        if self.tokenizer.eos_token:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        else:
            pad_token = '[PAD]'
            self.tokenizer.add_special_tokens({'pad_token': pad_token})
            self.tokenizer.pad_token = pad_token

        for param in self.llm_model.parameters():
            param.requires_grad = False

        if configs.prompt_domain:
            self.description = configs.content
        else:
            #self.description = 'The Electricity Transformer Temperature (ETT) is a crucial indicator in the electric power long-term deployment.'
            self.description = 'Weekly weather data is a crucial indicator in pest prediction models, crucial for effective pest management strategies.'

        self.dropout = nn.Dropout(configs.dropout)

        self.patch_embedding = PatchEmbedding(
            configs.d_model, self.patch_len, self.stride, configs.dropout)

        self.word_embeddings = self.llm_model.get_input_embeddings().weight
        self.vocab_size = self.word_embeddings.shape[0]
        self.num_tokens = 1000
        self.mapping_layer = nn.Linear(self.vocab_size, self.num_tokens)

        self.reprogramming_layer = ReprogrammingLayer(configs.d_model, configs.n_heads, self.d_ff, self.d_llm)

        self.patch_nums = int((configs.seq_len - self.patch_len) / self.stride + 2)
        self.head_nf = self.d_ff * self.patch_nums

        if self.task_name == 'long_term_forecast' or self.task_name == 'short_term_forecast':
            self.output_projection = FlattenHead(configs.enc_in, self.head_nf, self.pred_len,
                                                 head_dropout=configs.dropout)
        else:
            raise NotImplementedError

        self.normalize_layers = Normalize(configs.enc_in, affine=False)

    #def forward(self, x_enc, x_mark_enc, x_dec, x_mark_dec, mask=None):
    def forward(self, x_enc, mask=None):
        if self.task_name == 'long_term_forecast' or self.task_name == 'short_term_forecast':
            dec_out = self.forecast(x_enc)
            return dec_out[:, -self.pred_len:, :]
        return None

    #def forecast(self, x_enc, x_mark_enc, x_dec, x_mark_dec):
    def forecast(self, x_enc):
        x_enc = self.normalize_layers(x_enc, 'norm')

        B, T, N = x_enc.size()
        x_enc = x_enc.permute(0, 2, 1).contiguous().reshape(B * N, T, 1)

        min_values = torch.min(x_enc, dim=1)[0]
        max_values = torch.max(x_enc, dim=1)[0]
        medians = torch.median(x_enc, dim=1).values
        lags = self.calcute_lags(x_enc)
        trends = x_enc.diff(dim=1).sum(dim=1)

        prompt = []
        for b in range(x_enc.shape[0]):
            min_values_str = str(min_values[b].tolist()[0])
            max_values_str = str(max_values[b].tolist()[0])
            median_values_str = str(medians[b].tolist()[0])
            lags_values_str = str(lags[b].tolist())
            prompt_ = (
                f"<|start_prompt|>Dataset description: {self.description}"
                f"Task description: forecast the next {str(self.pred_len)} steps given the previous {str(self.seq_len)} steps information; "
                "Input statistics: "
                f"min value {min_values_str}, "
                f"max value {max_values_str}, "
                f"median value {median_values_str}, "
                f"the trend of input is {'upward' if trends[b] > 0 else 'downward'}, "
                f"top 5 lags are : {lags_values_str}<|<end_prompt>|>"
            )

            prompt.append(prompt_)

        x_enc = x_enc.reshape(B, N, T).permute(0, 2, 1).contiguous()

        prompt = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=2048).input_ids
        prompt_embeddings = self.llm_model.get_input_embeddings()(prompt.to(x_enc.device))  # (batch, prompt_token, dim)

        source_embeddings = self.mapping_layer(self.word_embeddings.permute(1, 0)).permute(1, 0)

        x_enc = x_enc.permute(0, 2, 1).contiguous()
        enc_out, n_vars = self.patch_embedding(x_enc)#.to(torch.bfloat16))
        enc_out = self.reprogramming_layer(enc_out, source_embeddings, source_embeddings)
        llama_enc_out = torch.cat([prompt_embeddings, enc_out], dim=1)
        dec_out = self.llm_model(inputs_embeds=llama_enc_out).last_hidden_state
        dec_out = dec_out[:, :, :self.d_ff]

        dec_out = torch.reshape(
            dec_out, (-1, n_vars, dec_out.shape[-2], dec_out.shape[-1]))
        dec_out = dec_out.permute(0, 1, 3, 2).contiguous()

        dec_out = self.output_projection(dec_out[:, :, :, -self.patch_nums:])
        dec_out = dec_out.permute(0, 2, 1).contiguous()

        dec_out = self.normalize_layers(dec_out, 'denorm')

        return dec_out

    def calcute_lags(self, x_enc):
        q_fft = torch.fft.rfft(x_enc.permute(0, 2, 1).contiguous(), dim=-1)
        k_fft = torch.fft.rfft(x_enc.permute(0, 2, 1).contiguous(), dim=-1)
        res = q_fft * torch.conj(k_fft)
        corr = torch.fft.irfft(res, dim=-1)
        mean_value = torch.mean(corr, dim=1)
        _, lags = torch.topk(mean_value, self.top_k, dim=-1)
        return lags


class ReprogrammingLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_keys=None, d_llm=None, attention_dropout=0.1):
        super(ReprogrammingLayer, self).__init__()

        d_keys = d_keys or (d_model // n_heads)

        self.query_projection = nn.Linear(d_model, d_keys * n_heads)
        self.key_projection = nn.Linear(d_llm, d_keys * n_heads)
        self.value_projection = nn.Linear(d_llm, d_keys * n_heads)
        self.out_projection = nn.Linear(d_keys * n_heads, d_llm)
        self.n_heads = n_heads
        self.dropout = nn.Dropout(attention_dropout)

    def forward(self, target_embedding, source_embedding, value_embedding):
        B, L, _ = target_embedding.shape
        S, _ = source_embedding.shape
        H = self.n_heads

        target_embedding = self.query_projection(target_embedding).view(B, L, H, -1)
        source_embedding = self.key_projection(source_embedding).view(S, H, -1)
        value_embedding = self.value_projection(value_embedding).view(S, H, -1)

        out = self.reprogramming(target_embedding, source_embedding, value_embedding)

        out = out.reshape(B, L, -1)

        return self.out_projection(out)

    def reprogramming(self, target_embedding, source_embedding, value_embedding):
        B, L, H, E = target_embedding.shape

        scale = 1. / sqrt(E)

        scores = torch.einsum("blhe,she->bhls", target_embedding, source_embedding)

        A = self.dropout(torch.softmax(scale * scores, dim=-1))
        reprogramming_embedding = torch.einsum("bhls,she->blhe", A, value_embedding)

        return reprogramming_embedding


def vali(args, model, vali_data, vali_loader, criterion, mae_metric, vis_flag):
    total_loss = []
    total_mae_loss = []
    model.eval()
    op = []
    pr = []
    with torch.no_grad():
        for i, (batch_x, batch_y) in tqdm(enumerate(vali_loader)):
            batch_x = batch_x.float()
            batch_y = batch_y.float()

            #batch_x_mark = batch_x_mark.float().to(accelerator.device)
            #batch_y_mark = batch_y_mark.float().to(accelerator.device)

            # decoder input
            dec_inp = torch.zeros_like(batch_y[:, -args.pred_len:, :]).float()
            dec_inp = torch.cat([batch_y[:, :args.label_len, :], dec_inp], dim=1).float()
            # encoder - decoder

            if args.output_attention:
                outputs = model(batch_x, dec_inp)[0]
            else:
                outputs = model(batch_x, dec_inp)

            #outputs, batch_y = accelerator.gather_for_metrics((outputs, batch_y))

            f_dim = -1 if args.features == 'MS' else 0
            outputs = outputs[:, -args.pred_len:, f_dim:]
            batch_y = batch_y[:, -args.pred_len:, f_dim:]

            pred = outputs.detach()
            true = batch_y.detach()
            #print(pred.shape)
        

            loss = criterion(pred, true)

            mae_loss = mae_metric(pred, true)

            total_loss.append(loss.item())
            total_mae_loss.append(mae_loss.item())

            if vis_flag:
                o1 = true[0][0]
                p1 = pred[0][0]
                for i in range(1,len(true)):
                    o1 = torch.cat((o1,true[i][0][-1].unsqueeze(0)))
                    p1 = torch.cat((p1,pred[i][0][-1].unsqueeze(0)))
                #op.append(vali_data.scaler.inverse_transform(o1.reshape(1,-1).cpu()))
                #pr.append(vali_data.scaler.inverse_transform(p1.reshape(1,-1).cpu())) 
                op.append(o1.reshape(1,-1).cpu())
                pr.append(p1.reshape(1,-1).cpu())  

    total_loss = np.average(total_loss)
    total_mae_loss = np.average(total_mae_loss)

    model.train()
    return total_loss, total_mae_loss, op, pr

def del_files(dir_path):
    shutil.rmtree(dir_path)


def adjust_learning_rate(optimizer, scheduler, epoch, args, printout=True):
    if args.lradj == 'type1':
        lr_adjust = {epoch: args.learning_rate * (0.5 ** ((epoch - 1) // 1))}
    elif args.lradj == 'type2':
        lr_adjust = {
            2: 5e-5, 4: 1e-5, 6: 5e-6, 8: 1e-6,
            10: 5e-7, 15: 1e-7, 20: 5e-8
        }
    elif args.lradj == 'type3':
        lr_adjust = {epoch: args.learning_rate if epoch < 3 else args.learning_rate * (0.9 ** ((epoch - 3) // 1))}
    elif args.lradj == 'PEMS':
        lr_adjust = {epoch: args.learning_rate * (0.95 ** (epoch // 1))}
    elif args.lradj == 'TST':
        lr_adjust = {epoch: scheduler.get_last_lr()[0]}
    elif args.lradj == 'constant':
        lr_adjust = {epoch: args.learning_rate}
    if epoch in lr_adjust.keys():
        lr = lr_adjust[epoch]
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        if printout:
            print('Updating learning rate to {}'.format(lr))

class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, save_mode=True):
        #self.accelerator = accelerator
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.save_mode = save_mode

    def __call__(self, val_loss, model, path):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            if self.save_mode:
                self.save_checkpoint(val_loss, model, path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')

            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            if self.save_mode:
                self.save_checkpoint(val_loss, model, path)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, path):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')


        torch.save(model.state_dict(), path + '/' + 'checkpoint')
        self.val_loss_min = val_loss


os.environ['CURL_CA_BUNDLE'] = ''
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

#from utils.tools import del_files, EarlyStopping, adjust_learning_rate, vali, load_content

parser = argparse.ArgumentParser(description='Time-LLM')

fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)


default_args = {
    'task_name': 'short_term_forecast',
    'is_training': 1,
    'model_id': 'test',
    'model_comment': 'none',
    'seed': 2021,
    'data': 'agri',
    'root_path': './',
    'data_path': 'raw_dat_v2.csv',
    'features': 'S',
    'target': 'RH1(%)',
    'loader': 'modal',
    'checkpoints': './checkpoints/',
    'seq_len': 48,
    'label_len': 24,
    'pred_len': 10,
    'enc_in': 7,
    'dec_in': 7,
    'c_out': 7,
    'd_model': 16,
    'n_heads': 8,
    'e_layers': 2,
    'd_layers': 1,
    'd_ff': 32,
    'moving_avg': 25,
    'factor': 1,
    'dropout': 0.1,
    'activation': 'gelu',
    'output_attention': False,
    'patch_len': 16,
    'stride': 8,
    'prompt_domain': 0,
    'llm_model': 'BERT',
    'llm_dim': 768, #4096 for Llama
    'num_workers': 10,
    'itr': 1,
    'train_epochs': 20,
    'align_epochs': 1,
    'batch_size': 32,
    'eval_batch_size': 1,
    'patience': 10,
    'learning_rate': 0.001,
    'des': 'test',
    'loss': 'MSE',
    'lradj': 'type1',
    'pct_start': 0.2,
    'use_amp': False,
    'llm_layers': 1,
    'percent': 100
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#args = parser.parse_args()
args = argparse.Namespace(**default_args)

for ii in range(args.itr):
    # setting record of experiments
    setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_{}_{}'.format(
        args.task_name,
        args.model_id,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        #args.embed,
        args.des, ii)

    train_data, train_loader = data_provider(args, 'train')
    vali_data, vali_loader = data_provider(args, 'val')
    test_data, test_loader = data_provider(args, 'test')

    '''if args.model == 'Autoformer':
        model = Autoformer.Model(args).float()
    elif args.model == 'DLinear':
        model = DLinear.Model(args).float()
    else:'''


    model = Model(args).float().to(device)

    train_loader = [(data.to(device), target.to(device)) for data, target in train_loader]
    test_loader = [(data.to(device), target.to(device)) for data, target in test_loader]
    vali_loader = [(data.to(device), target.to(device)) for data, target in vali_loader]

    path = os.path.join(args.checkpoints,
                        setting + '-' + args.model_comment)  # unique checkpoint saving path
    #args.content = load_content(args)
    if not os.path.exists(path):
        os.makedirs(path)

    time_now = time.time()

    train_steps = len(train_loader)
    early_stopping = EarlyStopping(patience=args.patience)

    trained_parameters = []
    for p in model.parameters():
        if p.requires_grad is True:
            trained_parameters.append(p)

    model_optim = optim.Adam(trained_parameters, lr=args.learning_rate)

    if args.lradj == 'COS':
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(model_optim, T_max=20, eta_min=1e-8)
    else:
        scheduler = lr_scheduler.OneCycleLR(optimizer=model_optim,
                                            steps_per_epoch=train_steps,
                                            pct_start=args.pct_start,
                                            epochs=args.train_epochs,
                                            max_lr=args.learning_rate)

    criterion = nn.MSELoss()
    mae_metric = nn.L1Loss()
    ops_rh = []
    prs_rh = []
    t_loss_rh = []
    v_loss_rh = []
    for epoch in range(args.train_epochs):
        iter_count = 0
        train_loss = []

        model.train()
        epoch_time = time.time()
        for i, (batch_x, batch_y) in tqdm(enumerate(train_loader)):
            iter_count += 1
            model_optim.zero_grad()

            batch_x = batch_x.float()
            batch_y = batch_y.float()
            #batch_x_mark = batch_x_mark.float()
            #batch_y_mark = batch_y_mark.float()

            dec_inp = torch.zeros_like(batch_y[:, -args.pred_len:, :]).float()
            dec_inp = torch.cat([batch_y[:, :args.label_len, :], dec_inp], dim=1).float()

            if args.output_attention:
                outputs = model(batch_x, dec_inp)[0]
            else:
                outputs = model(batch_x, dec_inp)


            f_dim = -1 if args.features == 'MS' else 0
            outputs = outputs[:, -args.pred_len:, f_dim:]
            batch_y = batch_y[:, -args.pred_len:, f_dim:]

            loss = criterion(outputs, batch_y)
            train_loss.append(loss.item())

            loss.backward()
            model_optim.step()

            if args.lradj == 'TST':
                adjust_learning_rate(model_optim, scheduler, epoch + 1, args)

        print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
        train_loss = np.average(train_loss)
        t_loss_rh.append(train_loss)

        vali_loss, vali_mae_loss,_,_ = vali(args, model, vali_data, vali_loader, criterion, mae_metric, vis_flag=False)
        v_loss_rh.append(vali_loss)

        if epoch == (args.train_epochs-1):
            vis_flag = True
            test_loss, test_mae_loss, ops_rh, prs_rh = vali(args, model, test_data, test_loader, criterion, mae_metric, vis_flag)
            f_loss_rh = (test_loss, test_mae_loss)
        else:
            vis_flag = False
            test_loss, test_mae_loss, _, _ = vali(args, model, test_data, test_loader, criterion, mae_metric, vis_flag)
        

        
        print(
            "Epoch: {0} | Train Loss: {1:.7f} Vali Loss: {2:.7f} Test Loss: {3:.7f} MAE Loss: {4:.7f}".format(
                epoch + 1, train_loss, vali_loss, test_loss, test_mae_loss))

        early_stopping(vali_loss, model, path)
        if early_stopping.early_stop:
            print("Early stopping")
            break

        if args.lradj != 'TST':
            if args.lradj == 'COS':
                scheduler.step()
                print("lr = {:.10f}".format(model_optim.param_groups[0]['lr']))
            else:
                if epoch == 0:
                    args.learning_rate = model_optim.param_groups[0]['lr']
                    print("lr = {:.10f}".format(model_optim.param_groups[0]['lr']))
                adjust_learning_rate(model_optim, scheduler, epoch + 1, args, printout=True)

        else:
            print('Updating learning rate to {}'.format(scheduler.get_last_lr()[0]))

if not os.path.exists('./checkpoints'):
    os.makedirs('./checkpoints')  # unique checkpoint saving path
del_files('./checkpoints')  # delete checkpoint files
print('success delete checkpoints')







25it [00:02,  8.50it/s]


Epoch: 1 cost time: 2.9422271251678467


3it [00:00, 14.91it/s]
7it [00:00, 17.92it/s]


Epoch: 1 | Train Loss: 0.9701965 Vali Loss: 0.4774393 Test Loss: 0.5988588 MAE Loss: 0.6146013
lr = 0.0000400000
Updating learning rate to 3.9999999999999996e-05


25it [00:02, 10.44it/s]


...

Epoch: 19 | Train Loss: 0.5236541 Vali Loss: 0.3056731 Test Loss: 0.3986943 MAE Loss: 0.4857966
Updating learning rate to 1.5258789062499999e-10


25it [00:02, 10.21it/s]


Epoch: 20 cost time: 2.449314594268799


3it [00:00, 14.47it/s]
7it [00:00, 17.68it/s]


Epoch: 20 | Train Loss: 0.5251916 Vali Loss: 0.3056731 Test Loss: 0.3986943 MAE Loss: 0.4857966
Updating learning rate to 7.629394531249999e-11
success delete checkpoints


In [4]:
o1_rh = ops_rh[0][0]
p1_rh = prs_rh[0][0]
for i in range(1,len(ops_rh)):
    o1_rh = np.concatenate((o1_rh,ops_rh[1][0]))
    p1_rh = np.concatenate((p1_rh,prs_rh[1][0]))

o2_rh = test_data.scaler.inverse_transform(o1_rh.reshape(-1,1))
p2_rh = test_data.scaler.inverse_transform(p1_rh.reshape(-1,1))

In [24]:
fig= plt.figure()

# Plot o1
plt.plot(o2_rh, label='truth')

# Plot p1
plt.plot(p2_rh, label='pred')

# Labeling and title
plt.xlabel('Week Index')
plt.ylabel('RH(%)')
plt.title('RH(%) Truth vs Prediction')

# Add legend
plt.legend()

# Save the plot as an image (e.g., PNG)
plt.savefig('plot_image_rh_20_epoch.png')

In [6]:
default_args = {
    'task_name': 'short_term_forecast',
    'is_training': 1,
    'model_id': 'test',
    'model_comment': 'none',
    'seed': 2021,
    'data': 'agri',
    'root_path': './',
    'data_path': 'raw_dat_v2.csv',
    'features': 'S',
    'target': 'MaxT(°C)',
    'loader': 'modal',
    'checkpoints': './checkpoints/',
    'seq_len': 48,
    'label_len': 24,
    'pred_len': 10,
    'enc_in': 7,
    'dec_in': 7,
    'c_out': 7,
    'd_model': 16,
    'n_heads': 8,
    'e_layers': 2,
    'd_layers': 1,
    'd_ff': 32,
    'moving_avg': 25,
    'factor': 1,
    'dropout': 0.1,
    'activation': 'gelu',
    'output_attention': False,
    'patch_len': 16,
    'stride': 8,
    'prompt_domain': 0,
    'llm_model': 'BERT',
    'llm_dim': 768, #4096 for Llama
    'num_workers': 10,
    'itr': 1,
    'train_epochs': 20,
    'align_epochs': 1,
    'batch_size': 32,
    'eval_batch_size': 1,
    'patience': 10,
    'learning_rate': 0.001,
    'des': 'test',
    'loss': 'MSE',
    'lradj': 'type1',
    'pct_start': 0.2,
    'use_amp': False,
    'llm_layers': 1,
    'percent': 100
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#args = parser.parse_args()
args = argparse.Namespace(**default_args)

for ii in range(args.itr):
    # setting record of experiments
    setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_{}_{}'.format(
        args.task_name,
        args.model_id,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        #args.embed,
        args.des, ii)

    train_data, train_loader = data_provider(args, 'train')
    vali_data, vali_loader = data_provider(args, 'val')
    test_data, test_loader = data_provider(args, 'test')

    '''if args.model == 'Autoformer':
        model = Autoformer.Model(args).float()
    elif args.model == 'DLinear':
        model = DLinear.Model(args).float()
    else:'''


    model = Model(args).float().to(device)

    train_loader = [(data.to(device), target.to(device)) for data, target in train_loader]
    test_loader = [(data.to(device), target.to(device)) for data, target in test_loader]
    vali_loader = [(data.to(device), target.to(device)) for data, target in vali_loader]

    path = os.path.join(args.checkpoints,
                        setting + '-' + args.model_comment)  # unique checkpoint saving path
    #args.content = load_content(args)
    if not os.path.exists(path):
        os.makedirs(path)

    time_now = time.time()

    train_steps = len(train_loader)
    early_stopping = EarlyStopping(patience=args.patience)

    trained_parameters = []
    for p in model.parameters():
        if p.requires_grad is True:
            trained_parameters.append(p)

    model_optim = optim.Adam(trained_parameters, lr=args.learning_rate)

    if args.lradj == 'COS':
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(model_optim, T_max=20, eta_min=1e-8)
    else:
        scheduler = lr_scheduler.OneCycleLR(optimizer=model_optim,
                                            steps_per_epoch=train_steps,
                                            pct_start=args.pct_start,
                                            epochs=args.train_epochs,
                                            max_lr=args.learning_rate)

    criterion = nn.MSELoss()
    mae_metric = nn.L1Loss()
    ops_maxt = []
    prs_maxt = []
    t_loss_maxt = []
    v_loss_maxt = []
    for epoch in range(args.train_epochs):
        iter_count = 0
        train_loss = []

        model.train()
        epoch_time = time.time()
        for i, (batch_x, batch_y) in tqdm(enumerate(train_loader)):
            iter_count += 1
            model_optim.zero_grad()

            batch_x = batch_x.float()
            batch_y = batch_y.float()
            #batch_x_mark = batch_x_mark.float()
            #batch_y_mark = batch_y_mark.float()

            dec_inp = torch.zeros_like(batch_y[:, -args.pred_len:, :]).float()
            dec_inp = torch.cat([batch_y[:, :args.label_len, :], dec_inp], dim=1).float()

            if args.output_attention:
                outputs = model(batch_x, dec_inp)[0]
            else:
                outputs = model(batch_x, dec_inp)


            f_dim = -1 if args.features == 'MS' else 0
            outputs = outputs[:, -args.pred_len:, f_dim:]
            batch_y = batch_y[:, -args.pred_len:, f_dim:]

            loss = criterion(outputs, batch_y)
            train_loss.append(loss.item())

            loss.backward()
            model_optim.step()

            if args.lradj == 'TST':
                adjust_learning_rate(model_optim, scheduler, epoch + 1, args)

        print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
        train_loss = np.average(train_loss)
        t_loss_maxt.append(train_loss)

        vali_loss, vali_mae_loss,_,_ = vali(args, model, vali_data, vali_loader, criterion, mae_metric, vis_flag=False)
        v_loss_maxt.append(vali_loss)
        if epoch == (args.train_epochs-1):
            vis_flag = True
            test_loss, test_mae_loss, ops_maxt, prs_maxt = vali(args, model, test_data, test_loader, criterion, mae_metric, vis_flag)
            f_loss_maxt = (test_loss, test_mae_loss)
        else:
            vis_flag = False
            test_loss, test_mae_loss, _, _ = vali(args, model, test_data, test_loader, criterion, mae_metric, vis_flag)
        

        

        
        print(
            "Epoch: {0} | Train Loss: {1:.7f} Vali Loss: {2:.7f} Test Loss: {3:.7f} MAE Loss: {4:.7f}".format(
                epoch + 1, train_loss, vali_loss, test_loss, test_mae_loss))

        early_stopping(vali_loss, model, path)
        if early_stopping.early_stop:
            print("Early stopping")
            break

        if args.lradj != 'TST':
            if args.lradj == 'COS':
                scheduler.step()
                print("lr = {:.10f}".format(model_optim.param_groups[0]['lr']))
            else:
                if epoch == 0:
                    args.learning_rate = model_optim.param_groups[0]['lr']
                    print("lr = {:.10f}".format(model_optim.param_groups[0]['lr']))
                adjust_learning_rate(model_optim, scheduler, epoch + 1, args, printout=True)

        else:
            print('Updating learning rate to {}'.format(scheduler.get_last_lr()[0]))

if not os.path.exists('./checkpoints'):
    os.makedirs('./checkpoints')  # unique checkpoint saving path
del_files('./checkpoints')  # delete checkpoint files
print('success delete checkpoints')


25it [00:02, 10.15it/s]


Epoch: 1 cost time: 2.465379476547241


3it [00:00, 14.48it/s]
7it [00:00, 17.89it/s]


Epoch: 1 | Train Loss: 1.1403683 Vali Loss: 0.8128854 Test Loss: 0.7543623 MAE Loss: 0.7133577
lr = 0.0000400000
Updating learning rate to 3.9999999999999996e-05


25it [00:02, 10.16it/s]


...

Epoch: 19 | Train Loss: 0.3890555 Vali Loss: 0.3177062 Test Loss: 0.2923925 MAE Loss: 0.4229423
Updating learning rate to 1.5258789062499999e-10


25it [00:02, 10.14it/s]


Epoch: 20 cost time: 2.46701979637146


3it [00:00, 14.53it/s]
7it [00:00, 17.23it/s]


Epoch: 20 | Train Loss: 0.3906893 Vali Loss: 0.3177062 Test Loss: 0.2923925 MAE Loss: 0.4229423
Updating learning rate to 7.629394531249999e-11
success delete checkpoints


In [7]:
o1_maxt = ops_maxt[0][0]
p1_maxt = prs_maxt[0][0]
for i in range(1,len(ops_maxt)):
    o1_maxt = np.concatenate((o1_maxt,ops_maxt[1][0]))
    p1_maxt = np.concatenate((p1_maxt,prs_maxt[1][0]))

o2_maxt = test_data.scaler.inverse_transform(o1_maxt.reshape(-1,1))
p2_maxt = test_data.scaler.inverse_transform(p1_maxt.reshape(-1,1))

In [23]:
fig= plt.figure()

# Plot o1
plt.plot(o2_maxt, label='Truth')

# Plot p1
plt.plot(p2_maxt, label='Pred')

# Labeling and title
plt.xlabel('Week Index')
plt.ylabel('MaxT(°C)')
plt.title('MaxT(°C) Truth vs Prediction')

# Add legend
plt.legend()

# Save the plot as an image (e.g., PNG)
plt.savefig('plot_image_maxT_20_epoch.png')

In [9]:
default_args = {
    'task_name': 'short_term_forecast',
    'is_training': 1,
    'model_id': 'test',
    'model_comment': 'none',
    'seed': 2021,
    'data': 'agri',
    'root_path': './',
    'data_path': 'raw_dat_v2.csv',
    'features': 'S',
    'target': 'RF(mm)',
    'loader': 'modal',
    'checkpoints': './checkpoints/',
    'seq_len': 48,
    'label_len': 24,
    'pred_len': 10,
    'enc_in': 7,
    'dec_in': 7,
    'c_out': 7,
    'd_model': 16,
    'n_heads': 8,
    'e_layers': 2,
    'd_layers': 1,
    'd_ff': 32,
    'moving_avg': 25,
    'factor': 1,
    'dropout': 0.1,
    'activation': 'gelu',
    'output_attention': False,
    'patch_len': 16,
    'stride': 8,
    'prompt_domain': 0,
    'llm_model': 'BERT',
    'llm_dim': 768, #4096 for Llama
    'num_workers': 10,
    'itr': 1,
    'train_epochs': 20,
    'align_epochs': 1,
    'batch_size': 32,
    'eval_batch_size': 1,
    'patience': 10,
    'learning_rate': 0.001,
    'des': 'test',
    'loss': 'MSE',
    'lradj': 'type1',
    'pct_start': 0.2,
    'use_amp': False,
    'llm_layers': 1,
    'percent': 100
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#args = parser.parse_args()
args = argparse.Namespace(**default_args)

for ii in range(args.itr):
    # setting record of experiments
    setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_{}_{}'.format(
        args.task_name,
        args.model_id,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        #args.embed,
        args.des, ii)

    train_data, train_loader = data_provider(args, 'train')
    vali_data, vali_loader = data_provider(args, 'val')
    test_data, test_loader = data_provider(args, 'test')

    '''if args.model == 'Autoformer':
        model = Autoformer.Model(args).float()
    elif args.model == 'DLinear':
        model = DLinear.Model(args).float()
    else:'''


    model = Model(args).float().to(device)

    train_loader = [(data.to(device), target.to(device)) for data, target in train_loader]
    test_loader = [(data.to(device), target.to(device)) for data, target in test_loader]
    vali_loader = [(data.to(device), target.to(device)) for data, target in vali_loader]

    path = os.path.join(args.checkpoints,
                        setting + '-' + args.model_comment)  # unique checkpoint saving path
    #args.content = load_content(args)
    if not os.path.exists(path):
        os.makedirs(path)

    time_now = time.time()

    train_steps = len(train_loader)
    early_stopping = EarlyStopping(patience=args.patience)

    trained_parameters = []
    for p in model.parameters():
        if p.requires_grad is True:
            trained_parameters.append(p)

    model_optim = optim.Adam(trained_parameters, lr=args.learning_rate)

    if args.lradj == 'COS':
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(model_optim, T_max=20, eta_min=1e-8)
    else:
        scheduler = lr_scheduler.OneCycleLR(optimizer=model_optim,
                                            steps_per_epoch=train_steps,
                                            pct_start=args.pct_start,
                                            epochs=args.train_epochs,
                                            max_lr=args.learning_rate)

    criterion = nn.MSELoss()
    mae_metric = nn.L1Loss()
    ops_rf = []
    prs_rf = []
    t_loss_rf = []
    v_loss_rf = []
    for epoch in range(args.train_epochs):
        iter_count = 0
        train_loss = []

        model.train()
        epoch_time = time.time()
        for i, (batch_x, batch_y) in tqdm(enumerate(train_loader)):
            iter_count += 1
            model_optim.zero_grad()

            batch_x = batch_x.float()
            batch_y = batch_y.float()
            #batch_x_mark = batch_x_mark.float()
            #batch_y_mark = batch_y_mark.float()

            dec_inp = torch.zeros_like(batch_y[:, -args.pred_len:, :]).float()
            dec_inp = torch.cat([batch_y[:, :args.label_len, :], dec_inp], dim=1).float()

            if args.output_attention:
                outputs = model(batch_x, dec_inp)[0]
            else:
                outputs = model(batch_x, dec_inp)


            f_dim = -1 if args.features == 'MS' else 0
            outputs = outputs[:, -args.pred_len:, f_dim:]
            batch_y = batch_y[:, -args.pred_len:, f_dim:]

            loss = criterion(outputs, batch_y)
            train_loss.append(loss.item())

            loss.backward()
            model_optim.step()

            if args.lradj == 'TST':
                adjust_learning_rate(model_optim, scheduler, epoch + 1, args)

        print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
        train_loss = np.average(train_loss)
        t_loss_rf.append(train_loss)

        vali_loss, vali_mae_loss,_,_ = vali(args, model, vali_data, vali_loader, criterion, mae_metric, vis_flag=False)
        v_loss_rf.append(vali_loss)

        if epoch == (args.train_epochs-1):
            vis_flag = True
            test_loss, test_mae_loss, ops_rf, prs_rf = vali(args, model, test_data, test_loader, criterion, mae_metric, vis_flag)
            f_loss_rf = (test_loss, test_mae_loss)
        else:
            vis_flag = False
            test_loss, test_mae_loss, _, _ = vali(args, model, test_data, test_loader, criterion, mae_metric, vis_flag)
        

        
        print(
            "Epoch: {0} | Train Loss: {1:.7f} Vali Loss: {2:.7f} Test Loss: {3:.7f} MAE Loss: {4:.7f}".format(
                epoch + 1, train_loss, vali_loss, test_loss, test_mae_loss))

        early_stopping(vali_loss, model, path)
        if early_stopping.early_stop:
            print("Early stopping")
            break

        if args.lradj != 'TST':
            if args.lradj == 'COS':
                scheduler.step()
                print("lr = {:.10f}".format(model_optim.param_groups[0]['lr']))
            else:
                if epoch == 0:
                    args.learning_rate = model_optim.param_groups[0]['lr']
                    print("lr = {:.10f}".format(model_optim.param_groups[0]['lr']))
                adjust_learning_rate(model_optim, scheduler, epoch + 1, args, printout=True)

        else:
            print('Updating learning rate to {}'.format(scheduler.get_last_lr()[0]))

if not os.path.exists('./checkpoints'):
    os.makedirs('./checkpoints')  # unique checkpoint saving path
del_files('./checkpoints')  # delete checkpoint files
print('success delete checkpoints')


25it [00:02, 10.11it/s]


Epoch: 1 cost time: 2.474858522415161


3it [00:00, 14.63it/s]
7it [00:00, 17.43it/s]


Epoch: 1 | Train Loss: 1.0829811 Vali Loss: 2.0068264 Test Loss: 1.2037178 MAE Loss: 0.8205271
lr = 0.0000400000
Updating learning rate to 3.9999999999999996e-05


25it [00:02, 10.15it/s]


...

Epoch: 19 | Train Loss: 0.9339080 Vali Loss: 1.8678251 Test Loss: 0.9801132 MAE Loss: 0.6821146
Updating learning rate to 1.5258789062499999e-10


25it [00:02, 10.16it/s]


Epoch: 20 cost time: 2.4620745182037354


3it [00:00, 14.38it/s]
7it [00:00, 17.00it/s]


Epoch: 20 | Train Loss: 0.9268297 Vali Loss: 1.8678250 Test Loss: 0.9801132 MAE Loss: 0.6821146
Updating learning rate to 7.629394531249999e-11
success delete checkpoints


In [10]:
o1_rf = ops_rf[0][0]
p1_rf = prs_rf[0][0]
for i in range(1,len(ops_rf)):
    o1_rf = np.concatenate((o1_rf,ops_rf[1][0]))
    p1_rf = np.concatenate((p1_rf,prs_rf[1][0]))

o2_rf = test_data.scaler.inverse_transform(o1_rf.reshape(-1,1))
p2_rf = test_data.scaler.inverse_transform(p1_rf.reshape(-1,1))

In [22]:
fig= plt.figure()

# Plot o1
plt.plot(o2_rf, label='Truth')

# Plot p1
plt.plot(p2_rf, label='Pred')

# Labeling and title
plt.xlabel('Week Index')
plt.ylabel('RF(mm)')
plt.title('RF(mm) Truth vs Prediction')

# Add legend
plt.legend()

# Save the plot as an image (e.g., PNG)
plt.savefig('plot_image_rf_20_epoch.png')

In [12]:
default_args = {
    'task_name': 'short_term_forecast',
    'is_training': 1,
    'model_id': 'test',
    'model_comment': 'none',
    'seed': 2021,
    'data': 'agri',
    'root_path': './',
    'data_path': 'raw_dat_v2.csv',
    'features': 'S',
    'target': 'MinT(°C)',
    'loader': 'modal',
    'checkpoints': './checkpoints/',
    'seq_len': 48,
    'label_len': 24,
    'pred_len': 10,
    'enc_in': 7,
    'dec_in': 7,
    'c_out': 7,
    'd_model': 16,
    'n_heads': 8,
    'e_layers': 2,
    'd_layers': 1,
    'd_ff': 32,
    'moving_avg': 25,
    'factor': 1,
    'dropout': 0.1,
    'activation': 'gelu',
    'output_attention': False,
    'patch_len': 16,
    'stride': 8,
    'prompt_domain': 0,
    'llm_model': 'BERT',
    'llm_dim': 768, #4096 for Llama
    'num_workers': 10,
    'itr': 1,
    'train_epochs': 20,
    'align_epochs': 1,
    'batch_size': 32,
    'eval_batch_size': 1,
    'patience': 10,
    'learning_rate': 0.001,
    'des': 'test',
    'loss': 'MSE',
    'lradj': 'type1',
    'pct_start': 0.2,
    'use_amp': False,
    'llm_layers': 1,
    'percent': 100
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#args = parser.parse_args()
args = argparse.Namespace(**default_args)

for ii in range(args.itr):
    # setting record of experiments
    setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_{}_{}'.format(
        args.task_name,
        args.model_id,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        #args.embed,
        args.des, ii)

    train_data, train_loader = data_provider(args, 'train')
    vali_data, vali_loader = data_provider(args, 'val')
    test_data, test_loader = data_provider(args, 'test')

    '''if args.model == 'Autoformer':
        model = Autoformer.Model(args).float()
    elif args.model == 'DLinear':
        model = DLinear.Model(args).float()
    else:'''


    model = Model(args).float().to(device)

    train_loader = [(data.to(device), target.to(device)) for data, target in train_loader]
    test_loader = [(data.to(device), target.to(device)) for data, target in test_loader]
    vali_loader = [(data.to(device), target.to(device)) for data, target in vali_loader]

    path = os.path.join(args.checkpoints,
                        setting + '-' + args.model_comment)  # unique checkpoint saving path
    #args.content = load_content(args)
    if not os.path.exists(path):
        os.makedirs(path)

    time_now = time.time()

    train_steps = len(train_loader)
    early_stopping = EarlyStopping(patience=args.patience)

    trained_parameters = []
    for p in model.parameters():
        if p.requires_grad is True:
            trained_parameters.append(p)

    model_optim = optim.Adam(trained_parameters, lr=args.learning_rate)

    if args.lradj == 'COS':
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(model_optim, T_max=20, eta_min=1e-8)
    else:
        scheduler = lr_scheduler.OneCycleLR(optimizer=model_optim,
                                            steps_per_epoch=train_steps,
                                            pct_start=args.pct_start,
                                            epochs=args.train_epochs,
                                            max_lr=args.learning_rate)

    criterion = nn.MSELoss()
    mae_metric = nn.L1Loss()
    ops_mint = []
    prs_mint = []
    t_loss_mint = []
    v_loss_mint = []
    for epoch in range(args.train_epochs):
        iter_count = 0
        train_loss = []

        model.train()
        epoch_time = time.time()
        for i, (batch_x, batch_y) in tqdm(enumerate(train_loader)):
            iter_count += 1
            model_optim.zero_grad()

            batch_x = batch_x.float()
            batch_y = batch_y.float()
            #batch_x_mark = batch_x_mark.float()
            #batch_y_mark = batch_y_mark.float()

            dec_inp = torch.zeros_like(batch_y[:, -args.pred_len:, :]).float()
            dec_inp = torch.cat([batch_y[:, :args.label_len, :], dec_inp], dim=1).float()

            if args.output_attention:
                outputs = model(batch_x, dec_inp)[0]
            else:
                outputs = model(batch_x, dec_inp)


            f_dim = -1 if args.features == 'MS' else 0
            outputs = outputs[:, -args.pred_len:, f_dim:]
            batch_y = batch_y[:, -args.pred_len:, f_dim:]

            loss = criterion(outputs, batch_y)
            train_loss.append(loss.item())

            loss.backward()
            model_optim.step()

            if args.lradj == 'TST':
                adjust_learning_rate(model_optim, scheduler, epoch + 1, args)

        print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
        train_loss = np.average(train_loss)
        t_loss_mint.append(train_loss)

        vali_loss, vali_mae_loss,_,_ = vali(args, model, vali_data, vali_loader, criterion, mae_metric, vis_flag=False)
        v_loss_mint.append(vali_loss)

        if epoch == (args.train_epochs-1):
            vis_flag = True
            test_loss, test_mae_loss, ops_mint, prs_mint = vali(args, model, test_data, test_loader, criterion, mae_metric, vis_flag)
            f_loss_mint = (test_loss, test_mae_loss)
        else:
            vis_flag = False
            test_loss, test_mae_loss, _, _ = vali(args, model, test_data, test_loader, criterion, mae_metric, vis_flag)
        

        
        print(
            "Epoch: {0} | Train Loss: {1:.7f} Vali Loss: {2:.7f} Test Loss: {3:.7f} MAE Loss: {4:.7f}".format(
                epoch + 1, train_loss, vali_loss, test_loss, test_mae_loss))

        early_stopping(vali_loss, model, path)
        if early_stopping.early_stop:
            print("Early stopping")
            break

        if args.lradj != 'TST':
            if args.lradj == 'COS':
                scheduler.step()
                print("lr = {:.10f}".format(model_optim.param_groups[0]['lr']))
            else:
                if epoch == 0:
                    args.learning_rate = model_optim.param_groups[0]['lr']
                    print("lr = {:.10f}".format(model_optim.param_groups[0]['lr']))
                adjust_learning_rate(model_optim, scheduler, epoch + 1, args, printout=True)

        else:
            print('Updating learning rate to {}'.format(scheduler.get_last_lr()[0]))

if not os.path.exists('./checkpoints'):
    os.makedirs('./checkpoints')  # unique checkpoint saving path
del_files('./checkpoints')  # delete checkpoint files
print('success delete checkpoints')


25it [00:02, 10.16it/s]


Epoch: 1 cost time: 2.4626152515411377


3it [00:00, 14.36it/s]
7it [00:00, 17.44it/s]


Epoch: 1 | Train Loss: 1.1619161 Vali Loss: 1.1446088 Test Loss: 1.2154221 MAE Loss: 0.8905139
lr = 0.0000400000
Updating learning rate to 3.9999999999999996e-05


25it [00:02, 10.21it/s]


...

Epoch: 19 | Train Loss: 0.3446213 Vali Loss: 0.2317133 Test Loss: 0.3162072 MAE Loss: 0.4228000
Updating learning rate to 1.5258789062499999e-10


25it [00:02, 10.21it/s]


Epoch: 20 cost time: 2.4510536193847656


3it [00:00, 14.26it/s]
7it [00:00, 17.44it/s]


Epoch: 20 | Train Loss: 0.3485186 Vali Loss: 0.2317133 Test Loss: 0.3162072 MAE Loss: 0.4228000
Updating learning rate to 7.629394531249999e-11
success delete checkpoints


In [13]:
o1_mint = ops_mint[0][0]
p1_mint = prs_mint[0][0]
for i in range(1,len(ops_mint)):
    o1_mint = np.concatenate((o1_mint,ops_mint[1][0]))
    p1_mint = np.concatenate((p1_mint,prs_mint[1][0]))

o2_mint = test_data.scaler.inverse_transform(o1_mint.reshape(-1,1))
p2_mint = test_data.scaler.inverse_transform(p1_mint.reshape(-1,1))

In [21]:
fig= plt.figure()

# Plot o1
plt.plot(o2_mint, label='Truth')

# Plot p1
plt.plot(p2_mint, label='Pred')

# Labeling and title
plt.xlabel('Week Index')
plt.ylabel('MinT(°C)')
plt.title('MinT(°C)Truth vs Pred')

# Add legend
plt.legend()

# Save the plot as an image (e.g., PNG)
plt.savefig('plot_image_minT_20_epoch.png')

In [16]:
fig, axs = plt.subplots(2, 2)

axs[0,0].plot(t_loss_rh, label='train')
axs[0,0].plot(v_loss_rh, label='val')
axs[0,0].set_title('RH(%)')
axs[0,0].set_xlabel('Epoch')
axs[0,0].set_ylabel('MSE Loss')

axs[0,1].plot(t_loss_maxt, label='train')
axs[0,1].plot(v_loss_maxt, label='val')
axs[0,1].set_title('MaxT(°C)')
axs[0,1].set_xlabel('Epoch')
axs[0,1].set_ylabel('MSE Loss')

axs[1,0].plot(t_loss_mint, label='train')
axs[1,0].plot(v_loss_mint, label='val')
axs[1,0].set_title('MinT(°C)')
axs[1,0].set_xlabel('Epoch')
axs[1,0].set_ylabel('MSE Loss')


axs[1,1].plot(t_loss_rf, label='train')
axs[1,1].plot(v_loss_rf, label='val')
axs[1,1].set_title('RF(mm)')
axs[1,1].set_xlabel('Epoch')
axs[1,1].set_ylabel('MSE Loss')

fig.suptitle('Train, Val Loss vs Epoch', fontsize=16)

plt.tight_layout()
plt.savefig('epoch_loss.png')

In [20]:
fig= plt.figure()

f_loss = [f_loss_rh, f_loss_maxt, f_loss_mint, f_loss_rf]

mse_loss = [i[0] for i in f_loss]
mae_loss = [i[1] for i in f_loss]
features = ['RH(%)', 'MaxT(°C)', 'MinT(°C)', 'RF(mm)']

# Set the width of the bars
bar_width = 0.35

# Set the position of the bars on the x-axis
r1 = np.arange(len(mse_loss))
r2 = [x + bar_width for x in r1]

# Create the bar plot
plt.bar(r1, mse_loss, color='b', width=bar_width, edgecolor='grey', label='MSE Loss')
plt.bar(r2, mae_loss, color='r', width=bar_width, edgecolor='grey', label='MAE Loss')

# Add xticks on the middle of the group bars
plt.xlabel('Features', fontweight='bold')
plt.xticks([r + bar_width/2 for r in range(len(mse_loss))], features)

# Add a legend
plt.legend()

# Show the plot
plt.title('Comparison of MSE Loss and MAE Loss for Different Parameters')
plt.savefig('test_loss.png')
